In [1]:
import pandas as pd
import numpy as np
import nba_api
import json
import requests
import copy
import time
import scipy.io
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF

In [2]:
from nba_api.stats.endpoints import shotchartdetail

def get_player_shot_chart_data(playerID, teamID, season):
	response = shotchartdetail.ShotChartDetail(
		team_id=teamID,
		player_id=playerID,
		context_measure_simple = 'FGA',
		season_nullable=season,
		season_type_all_star='Regular Season'
	)

	content = json.loads(response.get_json())
	# transform contents into dataframe
	results = content['resultSets'][0]
	headers = results['headers']
	rows = results['rowSet']
	df = pd.DataFrame(rows)
	df.columns = headers

	return df

In [3]:
initial_shot_zones = {('Center(C)', 'Less Than 8 ft.'): 0,
                    ('Center(C)', '8-16 ft.'): 0,
                    ('Center(C)', '16-24 ft.'): 0,
                    ('Center(C)', '24+ ft.'): 0,
                    ('Left Side(L)', '8-16 ft.'): 0,
                    ('Left Side(L)', '16-24 ft.'): 0,
                    ('Left Side(L)', '24+ ft.'): 0,
                    ('Left Side Center(LC)', '16-24 ft.'): 0,
                    ('Left Side Center(LC)', '24+ ft.'): 0,
                    ('Right Side(R)', '8-16 ft.'): 0,
                    ('Right Side(R)', '16-24 ft.'): 0,
                    ('Right Side(R)', '24+ ft.'): 0,
                    ('Right Side Center(RC)', '16-24 ft.'): 0,
                    ('Right Side Center(RC)', '24+ ft.'): 0
                    }

## Spatial Shooting Data
#### (Needs to be re-run to have updated entries that are not the percentzges of each zone, but rather the relative amount of shots a player shoots from that zone)

In [4]:
# OLD METHOD: RUN get_shot_zone_freqs INSTEAD
def get_player_shot_pcts(playerID, teamID, season):
  df = get_player_shot_chart_data(playerID, teamID, season)

  shot_zone_totals = copy.deepcopy(initial_shot_zones)
  shot_zone_makes = copy.deepcopy(initial_shot_zones)
  
  for i in range(len(df)):
    zone_tuple = (df['SHOT_ZONE_AREA'][i], df['SHOT_ZONE_RANGE'][i])
    if df['SHOT_ZONE_AREA'][i] == 'Back Court(BC)':
      continue
    
    shot_zone_totals[zone_tuple] += 1

    if df['SHOT_MADE_FLAG'][i] == 1:
      shot_zone_makes[zone_tuple] += 1

  shot_zone_pcts = {}

  for zone in shot_zone_totals:
      if shot_zone_totals[zone] == 0:
        shot_zone_pcts[zone] = 0
      else:
        shot_zone_pcts[zone] = round((shot_zone_makes[zone]/shot_zone_totals[zone]), 3)
        
  return list(shot_zone_pcts.values())

In [5]:
def get_shot_zone_freqs(playerID, teamID, season):
  df = get_player_shot_chart_data(playerID, teamID, season)

  shot_zone_totals = copy.deepcopy(initial_shot_zones)
  
  n = len(df)
  for i in range(n):
    zone_tuple = (df['SHOT_ZONE_AREA'][i], df['SHOT_ZONE_RANGE'][i])
    if df['SHOT_ZONE_AREA'][i] == 'Back Court(BC)':
      continue
    shot_zone_totals[zone_tuple] += 1

  shot_zone_freqs = {}

  for zone in shot_zone_totals:
      shot_zone_freqs[zone] = round(shot_zone_totals[zone] / n, 3)
  return list(shot_zone_freqs.values())

#### Test with Steph Curry during the 2015-16 season

In [118]:
get_shot_zone_freqs(201939, 1610612744, '2015-16')

[0.289,
 0.031,
 0.01,
 0.128,
 0.017,
 0.016,
 0.039,
 0.014,
 0.168,
 0.02,
 0.014,
 0.033,
 0.035,
 0.178]

In [6]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.static import teams

#### ONLY RUN THIS CELL IF YOU WANT NEW DATA, OTHERWISE REREAD OLD DATA FROM CSV (SEE BELOW)

In [7]:
all_zone_freqs = {}
team_list = teams.get_teams()

for team in team_list:

  roster = commonteamroster.CommonTeamRoster(season='2021-22',
                                              team_id=team['id'],
                                              league_id_nullable='00')
  df = roster.common_team_roster.get_data_frame()
  for i in range(len(df.index)):
    player_id = df['PLAYER_ID'][i]
    player_team = df['TeamID'][i]
    
    print(player_id)
    try:
      player_zone_freqs = get_shot_zone_freqs(player_id, player_team, '2021-22')
      time.sleep(0.3) # time delay so that the NBA website doesn't get mad and lock us out for accessing data too quickly
      
      all_zone_freqs[player_id] = player_zone_freqs
      
    except:
      continue

all_zone_freqs_df = pd.DataFrame.from_dict(all_zone_freqs)

all_zone_freqs_df

1626153
1630552
1630536
1628989
1630219
1628995
101150
1627789
201568
203476
1629027
1629631
203992
203991
1630168
1628381
1630602
1630346
1628369
1627759
1630238
1628401
1630202
1629684
203917
1629752
1630174
1628464
1630573
203935
1628436
201143
1629057
1630271
201567
1629164
200765
1629012
1627747
1630596
1630612
1629650
1630205
1629685
1629636
1626224
202334
1628374
1628386
1629731
1630171
1629627
1629607
203468
1628984
1630529
1630230
1626195
1629637
1629109
1630184
1627742
1630631
202685
203503
1626204
1630530
202066
1629632
1628366
202684
1627884
1627936
1628972
203897
202696
201942
1630245
1628396
1630792
1630250
1629744
1629750
1629076
1630172
1628425
1629033
203504
1630589
203939
1630182
1627827
203501
1628973
1628373
203493
203915
1627737
1628467
202722
1626246
1629029
1630210
203210
1629008
1630538
202326
203115
1627854
1630267
1628432
1628420
203999
1630192
203085
1627750
1628427
201145
203932
1630228
1627780
1627814
1629185
1629673
1630541
1626172
202357
2738
202691
16296

,1626153,1630552,1630536,1628989,1630219,1628995,101150,1627789,201568,203476,...,1626149,1628998,1626162,1630208,202330,1630550,1630539,203486,1629023,1629083
0,0.428,0.561,0.571,0.195,0.516,0.289,0.244,0.280,0.168,0.196,...,0.901,0.515,0.335,0.956,0.353,0.364,0.714,0.938,0.406,0.0
1,0.048,0.024,0.000,0.112,0.031,0.067,0.119,0.022,0.069,0.000,...,0.023,0.017,0.030,0.011,0.110,0.073,0.071,0.034,0.006,0.0
2,0.007,0.000,0.000,0.022,0.016,0.000,0.047,0.005,0.023,0.018,...,0.000,0.012,0.003,0.000,0.024,0.000,0.000,0.000,0.002,0.0
3,0.071,0.073,0.000,0.066,0.016,0.022,0.044,0.038,0.111,0.268,...,0.006,0.045,0.092,0.000,0.071,0.036,0.000,0.000,0.184,0.0
4,0.030,0.049,0.000,0.026,0.000,0.000,0.094,0.016,0.065,0.062,...,0.012,0.012,0.012,0.000,0.028,0.018,0.000,0.009,0.008,0.0
5,0.011,0.000,0.000,0.011,0.000,0.000,0.034,0.005,0.028,0.009,...,0.000,0.005,0.003,0.000,0.006,0.000,0.071,0.000,0.000,0.0
6,0.045,0.049,0.000,0.063,0.141,0.111,0.081,0.113,0.028,0.036,...,0.000,0.130,0.079,0.000,0.039,0.055,0.071,0.000,0.068,0.0
7,0.007,0.024,0.000,0.025,0.000,0.000,0.062,0.000,0.057,0.027,...,0.017,0.019,0.001,0.000,0.026,0.000,0.000,0.000,0.000,0.0
8,0.145,0.073,0.143,0.164,0.188,0.111,0.100,0.156,0.174,0.223,...,0.006,0.052,0.127,0.000,0.096,0.200,0.071,0.000,0.137,0.0
9,0.022,0.000,0.000,0.047,0.000,0.000,0.062,0.011,0.059,0.036,...,0.023,0.028,0.017,0.033,0.055,0.000,0.000,0.009,0.013,0.0


In [8]:
all_zone_freqs_df.to_csv('Math_189_PreNMF_Spatial_Data.csv', index=False)  

In [10]:
reread_data = pd.read_csv('Math_189_PreNMF_Spatial_Data.csv')

In [11]:
reread_data.drop(['1630792',
                '1629875',
                '1629623',
                '1628221',
                '1629619',
                '1630994',
                '1630296'], axis=1)

,1626153,1630552,1630536,1628989,1630219,1628995,101150,1627789,201568,203476,...,1626149,1628998,1626162,1630208,202330,1630550,1630539,203486,1629023,1629083
0,0.574,0.826,0.25,0.568,0.636,0.692,0.423,0.481,0.610,0.636,...,0.697,0.592,0.622,0.651,0.541,0.650,0.8,0.667,0.616,-1.0
1,0.308,0.000,-1.00,0.541,0.000,0.333,0.395,0.250,0.512,-1.000,...,0.000,0.143,0.464,1.000,0.353,0.500,0.0,0.091,0.333,-1.0
2,1.000,-1.000,-1.00,0.412,1.000,-1.000,0.333,0.000,0.286,0.500,...,-1.000,0.200,0.333,-1.000,0.667,-1.000,-1.0,-1.000,0.000,-1.0
3,0.368,0.333,-1.00,0.360,0.000,0.000,0.357,0.143,0.303,0.400,...,0.000,0.421,0.337,-1.000,0.341,0.000,-1.0,-1.000,0.388,-1.0
4,0.125,0.000,-1.00,0.400,-1.000,-1.000,0.433,0.667,0.385,0.571,...,0.500,0.400,0.273,-1.000,0.588,0.000,-1.0,0.667,0.500,-1.0
5,0.333,-1.000,-1.00,0.375,-1.000,-1.000,0.636,1.000,0.294,0.000,...,-1.000,0.500,0.333,-1.000,0.500,-1.000,0.0,-1.000,-1.000,-1.0
6,0.333,0.000,-1.00,0.479,0.222,0.200,0.462,0.667,0.471,0.250,...,-1.000,0.345,0.338,-1.000,0.500,0.333,1.0,-1.000,0.417,-1.0
7,0.500,0.000,-1.00,0.474,-1.000,-1.000,0.300,-1.000,0.382,0.333,...,0.000,0.250,0.000,-1.000,0.375,-1.000,-1.0,-1.000,-1.000,-1.0
8,0.462,0.333,0.00,0.360,0.417,0.000,0.281,0.276,0.375,0.440,...,0.000,0.409,0.347,-1.000,0.407,0.273,0.0,-1.000,0.411,-1.0
9,0.333,-1.000,-1.00,0.528,-1.000,-1.000,0.350,0.500,0.486,0.500,...,0.250,0.500,0.250,1.000,0.294,-1.000,-1.0,0.667,0.571,-1.0


In [12]:
shot_data = reread_data.replace({-1: 0})
np_shot_data = shot_data.to_numpy()

In [81]:
model = NMF(n_components=3, init='random', random_state=0,max_iter = 1000)  #learn the NMF model
W = model.fit_transform(np_shot_data)                                              #access the NMF factor matrices
H = model.components_

In [ ]:
H

In [82]:
playerIDs = shot_data.columns.values.tolist()

playerNames = []
for id in playerIDs:
    try:
        player_name = players.find_player_by_id(id)['full_name']
    except:
        if id == '1630314':
            player_name = 'Brandon Williams'
        if id == '1630322':
            player_name = 'Lindy Waters III'
    playerNames.append(player_name)

In [83]:
hidden_shots_df = pd.DataFrame(H, columns=playerNames)

## Yearly Stats Data

#### ONCE AGAIN, ONLY RUN THIS CELL IF YOU WANT NEW STATS DATA

In [20]:
from nba_api.stats.endpoints import playercareerstats
all_player_stats = {}
player_list = players.get_active_players()
count = 0
for player in player_list:
    try:
        career = playercareerstats.PlayerCareerStats(player_id=player['id'], per_mode36='Per36') 
    except:
        print(player['id'])
        continue 
    time.sleep(0.3) # time delay so that the NBA website doesn't get mad and lock us out for accessing data too quickly
    career_stats = career.get_data_frames()[0]
 
    for i in range(len(career_stats.index)):
        if career_stats['SEASON_ID'][i] == '2021-22':
            if career_stats['GP'][i] >= 10:
                important_stats = career_stats.iloc[i][['FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 'OREB', 'DREB', 'AST', 'STL', 'BLK']]
                
                important_stats['3/FG'] = important_stats['FG3A'] / important_stats['FGA']
                important_stats.drop(['FG3A'])

                all_player_stats[player['id']] = important_stats.tolist()
                count +=1 
                print(count)
            

all_player_stats_df = pd.DataFrame.from_dict(all_player_stats)

all_player_stats_df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,1630173,203500,1628389,1630583,200746,1629638,1628960,1628386,1630631,203937,...,1629685,1626159,1626174,1630218,1626153,201152,1629027,1630209,203469,1627826
0,12.600000,6.900,14.400000,13.200000,15.700000,16.700000,11.300000,10.800000,12.700000,11.40000,...,7.700000,11.900000,15.100000,13.600,6.700000,11.600000,21.000000,12.600000,9.100000,9.600
1,0.439000,0.547,0.557000,0.402000,0.550000,0.372000,0.448000,0.677000,0.446000,0.44600,...,0.378000,0.428000,0.501000,0.125,0.454000,0.518000,0.460000,0.526000,0.567000,0.626
2,3.300000,0.000,0.100000,4.800000,1.600000,8.300000,7.800000,0.200000,4.700000,2.60000,...,5.500000,3.100000,5.800000,3.400,2.900000,2.000000,8.300000,0.600000,0.400000,0.000
3,0.359000,0.000,0.000000,0.125000,0.304000,0.311000,0.409000,0.100000,0.291000,0.33000,...,0.300000,0.227000,0.390000,0.250,0.379000,0.354000,0.382000,0.091000,0.000000,0.000
4,3.000000,6.300,2.700000,3.300000,2.500000,0.900000,0.600000,3.800000,1.100000,1.70000,...,1.600000,2.300000,1.900000,4.300,1.200000,3.400000,0.700000,4.300000,5.100000,4.200
5,6.800000,7.400,8.400000,5.400000,6.300000,3.700000,3.800000,8.200000,3.200000,7.30000,...,5.300000,7.100000,9.900000,5.100,4.200000,5.500000,3.200000,10.700000,7.600000,8.300
6,1.700000,4.600,3.700000,2.100000,1.400000,3.800000,2.000000,1.800000,6.600000,4.40000,...,2.700000,3.900000,2.700000,2.600,4.700000,4.400000,10.000000,2.500000,2.200000,2.300
7,0.800000,1.200,1.600000,0.600000,0.500000,1.100000,0.900000,0.900000,3.100000,1.90000,...,1.100000,1.600000,0.900000,0.900,2.300000,2.300000,1.000000,0.900000,0.800000,0.700
8,0.900000,1.100,0.900000,1.000000,1.600000,0.600000,0.400000,1.500000,0.300000,1.10000,...,0.200000,1.200000,1.100000,0.900,0.500000,0.800000,0.100000,1.000000,0.600000,1.500
9,0.261905,0.000,0.006944,0.363636,0.101911,0.497006,0.690265,0.018519,0.370079,0.22807,...,0.714286,0.260504,0.384106,0.250,0.432836,0.172414,0.395238,0.047619,0.043956,0.000


In [23]:
all_player_stats_df.to_csv('Math_189_PreNMF_yearStats_Data.csv', index=False)  

In [15]:
all_player_stats_df = pd.read_csv('Math_189_PreNMF_yearStats_Data.csv')

In [16]:
np_stats_data = all_player_stats_df.to_numpy()

In [19]:
model = NMF(n_components=3, init='random', random_state=0,max_iter = 1000)  #learn the NMF model
W = model.fit_transform(np_stats_data)                                              #access the NMF factor matrices
H = model.components_

In [20]:
playerIDs = all_player_stats_df.columns.values.tolist()

playerNames = []
for id in playerIDs:
    try:
        player_name = players.find_player_by_id(id)['full_name']
    except:
        if id == '1630314':
            player_name = 'Brandon Williams'
        if id == '1630322':
            player_name = 'Lindy Waters III'
    playerNames.append(player_name)

In [21]:
hidden_stats_df = pd.DataFrame(H, columns=playerNames)

In [22]:
hidden_stats_df

,Precious Achiuwa,Steven Adams,Bam Adebayo,Santi Aldama,LaMarcus Aldridge,Nickeil Alexander-Walker,Grayson Allen,Jarrett Allen,Jose Alvarado,Kyle Anderson,...,Dylan Windler,Justise Winslow,Christian Wood,Robert Woodard II,Delon Wright,Thaddeus Young,Trae Young,Omer Yurtseven,Cody Zeller,Ivica Zubac
0,0.671507,0.000000,0.194383,0.910737,0.634157,1.564781,1.337653,0.105096,0.767404,0.399316,...,0.760801,0.504990,1.002823,0.706144,0.307218,0.343204,1.481795,0.127213,0.023097,0.000000
1,1.667449,1.867263,2.100802,1.312407,1.791687,0.427927,0.408607,2.226365,0.310610,1.460562,...,0.813296,1.503034,1.956742,1.443764,0.608772,1.347330,0.000000,2.706042,2.120767,2.222523
2,0.634858,0.835302,1.594292,0.679132,1.054109,1.220491,0.360591,0.737114,2.050834,1.328264,...,0.365433,1.169485,0.681426,0.984610,1.175532,1.463957,3.183701,0.868174,0.663062,0.740922


## RGB Analysis

In [23]:
normalized_hsd = hidden_stats_df.copy()

for col in normalized_hsd:
    normalized_hsd[col] = normalized_hsd[col]/normalized_hsd[col].sum()
    normalized_hsd[col] *= 255
    normalized_hsd[col] = normalized_hsd[col].astype(int) 
normalized_hsd

,Precious Achiuwa,Steven Adams,Bam Adebayo,Santi Aldama,LaMarcus Aldridge,Nickeil Alexander-Walker,Grayson Allen,Jarrett Allen,Jose Alvarado,Kyle Anderson,...,Dylan Windler,Justise Winslow,Christian Wood,Robert Woodard II,Delon Wright,Thaddeus Young,Trae Young,Omer Yurtseven,Cody Zeller,Ivica Zubac
0,57,0,12,80,46,124,161,8,62,31,...,100,40,70,57,37,27,80,8,2,0
1,142,176,137,115,131,33,49,185,25,116,...,106,120,137,117,74,108,0,186,192,191
2,54,78,104,59,77,96,43,61,167,106,...,48,93,47,80,143,118,174,59,60,63


In [24]:
normalized_hsd['Draymond Green']

0      0
1     90
2    164
Name: Draymond Green, dtype: int64

In [86]:
new_columns = normalized_hsd.columns[normalized_hsd.iloc[0].argsort()]

In [87]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
normalized_hsd[new_columns]

,Ivica Zubac,Devontae Cacok,Clint Capela,Kevon Looney,Nerlens Noel,Onyeka Okongwu,Tony Bradley,Mason Plumlee,Mitchell Robinson,DeAndre Jordan,Chris Silva,Andre Drummond,Jarred Vanderbilt,Draymond Green,Rudy Gobert,Jericho Sims,Hassan Whiteside,Robert Williams III,Tacko Fall,Steven Adams,Cody Zeller,Enes Kanter,Dwight Howard,Tyler Cook,Udoka Azubuike,Jakob Poeltl,Alize Johnson,Willie Cauley-Stein,Kris Dunn,Jarrett Allen,Omer Yurtseven,Isaiah Hartenstein,Nicolas Claxton,Daniel Gafford,Khem Birch,Charles Bassey,Moses Brown,Richaun Holmes,Tristan Thompson,Nick Richards,Bam Adebayo,Drew Eubanks,JaVale McGee,Willy Hernangomez,Tomas Satoransky,Isaiah Stewart,Jusuf Nurkic,Trent Forrest,Dennis Smith Jr.,RJ Nembhard Jr.,Montrezl Harrell,Bruno Fernando,Neemias Queta,Elfrid Payton,Tre Jones,Domantas Sabonis,Dwight Powell,Day'Ron Sharpe,Chris Paul,Dewayne Dedmon,Damian Jones,Mamadi Diakite,Deandre Ayton,Brandon Clarke,Marcus Garrett,T.J. McConnell,Paul Reed,Alex Len,Usman Garuba,Derrick Favors,Robin Lopez,Taj Gibson,Markelle Fultz,Nathan Knight,Thaddeus Young,Kai Jones,Trendon Watford,Nikola Jokic,Ish Smith,Zach Collins,Evan Mobley,Kyle Anderson,DeAndre' Bembry,Thanasis Antetokounmpo,Jonas Valanciunas,Udonis Haslem,Alperen Sengun,Larry Nance Jr.,Rajon Rondo,Dalano Banton,Bruce Brown,Jimmy Butler,Delon Wright,Bol Bol,Isaiah Jackson,Jordan McLaughlin,Anthony Davis,Giannis Antetokounmpo,Dejounte Murray,Jaxson Hayes,Russell Westbrook,Justise Winslow,James Johnson,Xavier Tillman Sr.,Vlatko Cancar,Terry Taylor,Juan Toscano-Anderson,Andre Iguodala,Josh Giddey,Keita Bates-Diop,Boban Marjanovic,Saben Lee,John Konchar,Pascal Siakam,Anthony Gill,Paul Millsap,Marquese Chriss,LaMarcus Aldridge,Joel Embiid,Scottie Barnes,Wendell Carter Jr.,DeMar DeRozan,David Nwaba,Killian Hayes,Leandro Bolmaro,Nemanja Bjelica,Raul Neto,Daishen Nix,Devon Dotson,Wenyen Gabriel,Cory Joseph,James Harden,Reggie Perry,Cody Martin,Al Horford,Jae'Sean Tate,Hamidou Diallo,Josh Hart,Javonte Green,Josh Green,Marvin Bagley III,Aaron Holiday,Nikola Vucevic,Robert Woodard II,Terance Mann,Tyrese Haliburton,Precious Achiuwa,Derrick Jones Jr.,Alex Caruso,Jalen Brunson,Ayo Dosunmu,Thomas Bryant,Eric Bledsoe,Isaac Bonga,Tyus Jones,Solomon Hill,Julius Randle,Ja Morant,Lamar Stevens,Brandon Ingram,Herbert Jones,Goran Dragic,P.J. Tucker,Isaiah Roby,Serge Ibaka,Jrue Holiday,Jose Alvarado,John Collins,Brodric Thomas,Greg Brown III,Kevin Durant,Kenrich Williams,Tobias Harris,Patrick Williams,Karl-Anthony Towns,Goga Bitadze,JaMychal Green,Cameron Payne,De'Aaron Fox,Kelly Olynyk,Justin Champagnie,Ricky Rubio,CJ Elleby,Gary Payton II,Abdel Nader,Christian Wood,Aaron Gordon,Bradley Beal,Kyle Lowry,Patrick Beverley,Daniel Theis,JT Thor,Malcolm Brogdon,Blake Griffin,Deni Avdija,Shai Gilgeous-Alexander,Dennis Schroder,Shake Milton,KJ Martin,Luka Doncic,Stanley Johnson,Jalen Smith,Franz Wagner,Aleksej Pokusevski,Jalen Suggs,Marcus Smart,Derrick Rose,Chris Chiozza,George Hill,Cade Cunningham,Mo Bamba,Spencer Dinwiddie,Naz Reid,Gorgui Dieng,Darius Garland,Talen Horton-Tucker,Jalen Johnson,Tyrese Maxey,Kyle Kuzma,Lou Williams,Luka Garza,Trey Lyles,R.J. Hampton,KZ Okpala,Malik Fitts,LaMelo Ball,Caris LeVert,Kristaps Porzingis,Jalen McDaniels,Cole Anthony,Chimezie Metu,Monte Morris,David Duke Jr.,Isaac Okoro,Santi Aldama,Trae Young,Gordon Hayward,Jabari Parker,Facundo Campazzo,Kevin Pangos,Obi Toppin,Jeff Green,Jonathan Kuminga,LeBron James,Bobby Portis,Derrick White,Miles Bridges,Naji Marshall,Troy Brown Jr.,Davon Reed,Otto Porter Jr.,Josh Christopher,Darius Bazley,PJ Dozier,Chris Boucher,Josh Okogie,Jarrett Culver,Jock Landale,Sharife Cooper,Caleb Martin,Khris Middleton,Austin Reaves,Skylar Mays,Mikal Bridges,Davion Mitchell,Myles Turner,Markieff Morris,Zeke Nnaji,Jaylen Nowell,Sandro Mamukelashvili,Royce O'Neale,Nassir Little,Sterling Brown,Rui Hachimura,Aaron Wiggins,Theo Maledon,Keon Johnson,Petr Cornelie,Harrison Barnes,Kevin Porter Jr.,Kira Lewis Jr.,Paul George,Jeremiah Robinson-Earl,Matisse Thybul

In [67]:
normalized_hsd[['Stephen Curry', 'Giannis Antetokounmpo', 'Nikola Jokic', 'LeBron James', 'Joel Embiid', 'Jayson Tatum', 'Kevin Durant', 'Klay Thompson', 'Luka Doncic', 'Andrew Wiggins']]

,Stephen Curry,Giannis Antetokounmpo,Nikola Jokic,LeBron James,Joel Embiid,Jayson Tatum,Kevin Durant,Klay Thompson,Luka Doncic,Andrew Wiggins
0,131,39,28,82,46,99,63,146,73,108
1,13,106,106,59,115,66,62,29,52,59
2,109,108,119,112,92,88,128,79,128,86


In [97]:
normalized_shots_df = hidden_shots_df.copy()

for col in normalized_shots_df:
    try:
        normalized_shots_df[col] = normalized_shots_df[col]/normalized_shots_df[col].sum()
        normalized_shots_df[col] *= 255
        normalized_shots_df[col] = normalized_shots_df[col].astype(int)
    except:
        continue

In [104]:
normalized_shots_df['Kevon Looney']

0    140
1    103
2     10
Name: Kevon Looney, dtype: int64

In [103]:
normalized_shots_df[['Stephen Curry', 'Giannis Antetokounmpo', 'Nikola Jokic', 'LeBron James', 'Joel Embiid', 'Jayson Tatum', 'Kevin Durant', 'Klay Thompson', 'Luka Doncic', 'Andrew Wiggins', 'Rudy Gobert', 'Chris Paul', 'Duncan Robinson']]

,Stephen Curry,Giannis Antetokounmpo,Nikola Jokic,LeBron James,Joel Embiid,Jayson Tatum,Kevin Durant,Klay Thompson,Luka Doncic,Andrew Wiggins,Rudy Gobert,Chris Paul,Duncan Robinson
0,92,92,95,83,83,91,109,69,88,99,0,89,5
1,89,80,81,85,95,88,80,83,109,94,255,73,90
2,73,82,77,86,75,75,64,102,56,60,0,92,159


In [102]:
sorted_cols = normalized_shots_df.columns[normalized_shots_df.iloc[1].argsort()]
normalized_shots_df[sorted_cols]

,Michael Porter Jr.,Skylar Mays,Dean Wade,Keita Bates-Diop,Robert Williams III,Matt Thomas,Tomas Satoransky,Ziaire Williams,Terance Mann,Terance Mann,JaMychal Green,Alex Len,Theo Pinson,Jordan Clarkson,Nah'Shon Hyland,Lauri Markkanen,Nickeil Alexander-Walker,Daniel Gafford,Chris Chiozza,Terry Rozier,Quentin Grimes,Cody Martin,Jake Layman,Justise Winslow,Miles Bridges,Troy Brown Jr.,Miles McBride,Jock Landale,Kessler Edwards,Donte DiVincenzo,Rodney McGruder,Rodney McGruder,Elijah Hughes,Delon Wright,Killian Tillie,Jarrett Allen,Caleb Martin,CJ McCollum,Tyus Jones,Ayo Dosunmu,Raul Neto,Tony Bradley,Tony Bradley,Cameron Payne,Aaron Nesmith,John Konchar,Markieff Morris,Dennis Schroder,Jonas Valanciunas,Lamar Stevens,Jakob Poeltl,Donovan Mitchell,Rui Hachimura,Robert Covington,Jimmy Butler,Jrue Holiday,Tre Jones,Sterling Brown,Aaron Holiday,Anthony Davis,David Duke Jr.,Joshua Primo,Thomas Bryant,Cam Thomas,Dillon Brooks,Ignas Brazdeikis,Ish Wainright,Robin Lopez,Tyrese Haliburton,Facundo Campazzo,Devin Booker,George Hill,Frank Jackson,Devonte' Graham,Devontae Cacok,Mo Bamba,Jaden McDaniels,Kyle Kuzma,Doug McDermott,Shai Gilgeous-Alexander,Yuta Watanabe,LaMarcus Aldridge,Russell Westbrook,Theo Maledon,JaVale McGee,Lonnie Walker IV,Trae Young,Seth Curry,Chris Boucher,David Johnson,Isaiah Stewart,Kentavious Caldwell-Pope,Talen Horton-Tucker,Luka Garza,Lou Williams,Reggie Bullock,Bogdan Bogdanovic,Avery Bradley,Giannis Antetokounmpo,Patty Mills,Kevin Durant,Jordan Poole,Carmelo Anthony,Darius Garland,Nikola Jokic,Myles Turner,Richaun Holmes,Jalen Green,Aaron Gordon,Gary Harris,Tim Hardaway Jr.,Jalen Brunson,Cameron Johnson,Klay Thompson,Marcus Morris Sr.,Frank Ntilikina,Neemias Queta,DeMarcus Cousins,DeMar DeRozan,Josh Jackson,Garrett Temple,Evan Fournier,Rudy Gobert,LeBron James,Terence Davis,Tre Mann,Steven Adams,Mikal Bridges,Monte Morris,Brandon Ingram,Paul George,Dejounte Murray,John Collins,Eric Paschall,Jayson Tatum,Stephen Curry,Vit Krejci,Lance Stephenson,Pascal Siakam,Bam Adebayo,Nate Hinton,Domantas Sabonis,Bobby Portis,Duncan Robinson,Brandon Clarke,Bradley Beal,Keon Johnson,Justin Holiday,Max Strus,DeAndre Jordan,Jordan Schakel,Nikola Vucevic,Khris Middleton,Draymond Green,Luguentz Dort,Payton Pritchard,Andrew Wiggins,Tyler Herro,Luke Kennard,Kemba Walker,Dewayne Dedmon,Jae Crowder,Jerami Grant,Fred VanVleet,Shake Milton,Jalen Smith,Isaiah Jackson,Aaron Wiggins,Santi Aldama,Saddiq Bey,Jared Harper,Mitchell Robinson,Udonis Haslem,Jaren Jackson Jr.,Serge Ibaka,Serge Ibaka,Jaylen Brown,Cassius Winston,Kristaps Porzingis,Immanuel Quickley,Davon Reed,Feron Hunt,Killian Hayes,Cedi Osman,Zach LaVine,Kyle Lowry,Kevin Huerter,Greg Brown III,RJ Barrett,Nicolas Batum,Armoni Brooks,Keldon Johnson,Derrick Favors,Collin Sexton,D'Angelo Russell,Zeke Nnaji,Bojan Bogdanovic,Derrick Rose,Duane Washington Jr.,Coby White,Rodney Hood,De'Andre Hunter,Will Barton,Austin Reaves,Carsen Edwards,Didi Louzada,Trent Forrest,Chuma Okeke,Anthony Edwards,Danny Green,Devin Cannady,Kevon Looney,Ivica Zubac,Boban Marjanovic,Wendell Carter Jr.,Eric Gordon,Romeo Langford,Otto Porter Jr.,Evan Mobley,Aleksej Pokusevski,Gabe Vincent,Jordan Nwora,Tony Bradley,Tony Bradley,Chris Duarte,Ryan Arcidiacono,Alex Caruso,Cade Cunningham,Saben Lee,Victor Oladipo,Jonathan Kuminga,Herbert Jones,Jaxson Hayes,CJ Elleby,Goga Bitadze,Josh Green,Nassir Little,Jay Scrubb,Al Horford,Moritz Wagner,Bismack Biyombo,Bismack Biyombo,Danilo Gallinari,Isaiah Thomas,Luka Doncic,Daniel Theis,Melvin Frazier,Brandon Boston Jr.,Khem Birch,Isaiah Roby,Jaylen Nowell,Trey Burke,Kyrie Irving,Jalen McDaniels,Grayson Allen,Jose Alvarado,Vernon Carey Jr.,Arnoldas Kulboka,Elfrid Payton,Karl-Anthony Towns,Terance Mann,Terance Mann,Kevin Love,Ty Jerome,Ja Morant,Reggie Jackson,Thanasis Antetokounmpo,Kevin Porter Jr.,Isaac Bonga,Corey Kispert,Jalen Suggs,Nemanja Bjelica,Ish Smith,Kelly Olynyk,Andre Iguodala,Naji Marshall,Alec Burks,Kelly Oubre Jr.,Malcolm Brogdon,James Bouknight,R.J. Hampton,Markus Howa

In [106]:
shot_weights_df = pd.DataFrame(W, columns=['Component 1', 'Component 2', 'Component 3'])
shot_weights_df

,Component 1,Component 2,Component 3
0,0.261157,1.002105,0.187605
1,0.305135,0.625543,0.085708
2,1.040065,0.000000,0.000000
3,0.141355,0.468937,0.217179
4,0.292940,0.345673,0.415280
5,0.000000,0.000000,0.941857
6,0.288529,0.482288,0.173745
7,0.000000,0.000000,1.252218
8,0.226935,0.454898,0.160270
9,0.104499,0.518693,0.393114


In [108]:
stats_weights_df = pd.DataFrame(W, columns=['Component 1', 'Component 2', 'Component 3'])
stats_weights_df

,Component 1,Component 2,Component 3
0,6.868584,3.232784,3.371828
1,0.161650,0.222535,0.088066
2,5.552025,0.000000,0.000000
3,0.208998,0.069104,0.052684
4,0.000000,1.778445,0.000000
5,1.288019,3.557329,0.540133
6,0.412328,0.000000,2.914414
7,0.416887,0.312904,0.408175
8,0.065603,0.709661,0.000000
9,0.410273,0.014555,0.000000
